In [ ]:
# !pip install git+https://github.com/openai/CLIP.git
# !pip install torch torchvision


In [2]:
import ssl
import os

# Désactive la vérification SSL pour le téléchargement
ssl._create_default_https_context = ssl._create_unverified_context
print("Certificats SSL contournés pour le téléchargement.")

Certificats SSL contournés pour le téléchargement.


### Initialisation 

In [3]:
import torch
import clip
from torchvision.datasets import Food101

# Configuration du device pour MacBook (M1, M2, M3)
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Device utilisé : {device}")

# C'est ici qu'on définit 'preprocess'
model, preprocess = clip.load("ViT-B/32", device=device)

Device utilisé : mps


100%|███████████████████████████████████████| 338M/338M [00:11<00:00, 30.6MiB/s]


### Téléchargement et Filtrage de Food-101

In [ ]:
from torchvision.datasets import Food101
import torch

# # 1. Téléchargement (seulement si nécessaire)
# full_train_ds = Food101(root="./data", split="train", download=True, transform=preprocess)
# full_test_ds = Food101(root="./data", split="test", download=True, transform=preprocess)

In [5]:
import os
import shutil
from torchvision.datasets import Food101, ImageFolder
from tqdm import tqdm

# Configuration des chemins
source_path = "./data"
filtered_path = "./data_filtered"
target_classes = ['pizza', 'hamburger', 'sushi', 'tacos', 'apple_pie', 
                  'ice_cream', 'omelette', 'french_fries', 'guacamole', 'paella']

def create_filtered_dataset(split, max_samples=200):
    split_path = os.path.join(filtered_path, split)
    
    if os.path.exists(split_path):
        print(f"✅ Dossier {split} déjà existant dans {filtered_path}")
        return

    print(f"📂 Création du dataset filtré ({split})...")
    # On charge le dataset original sans transformation pour copier les fichiers bruts
    full_ds = Food101(root=source_path, split=split, download=True)
    
    class_counts = {c: 0 for c in target_classes}
    target_indices = {full_ds.class_to_idx[c]: c for c in target_classes}

    for i in tqdm(range(len(full_ds))):
        _, label = full_ds[i]
        
        if label in target_indices:
            class_name = target_indices[label]
            if class_counts[class_name] < max_samples:
                # Créer le dossier de la classe
                os.makedirs(os.path.join(split_path, class_name), exist_ok=True)
                
                # Récupérer le chemin de l'image originale
                img_path = full_ds._image_files[i]
                dest_path = os.path.join(split_path, class_name, os.path.basename(img_path))
                
                # Copier le fichier
                shutil.copy(img_path, dest_path)
                class_counts[class_name] += 1
        
        if all(count >= max_samples for count in class_counts.values()):
            break

# Exécution pour Train et Test
create_filtered_dataset("train", max_samples=200)
create_filtered_dataset("test", max_samples=50)

📂 Création du dataset filtré (train)...


 95%|█████████▌| 72199/75750 [01:29<00:04, 810.92it/s]


📂 Création du dataset filtré (test)...


 95%|█████████▌| 24049/25250 [00:31<00:01, 766.64it/s]


### Chargement du Dataset filtré

In [6]:
from torchvision import transforms

# On utilise le 'preprocess' de CLIP comme transformation
train_ds = ImageFolder(root=os.path.join(filtered_path, "train"), transform=preprocess)
test_ds = ImageFolder(root=os.path.join(filtered_path, "test"), transform=preprocess)

print(f"Classes chargées : {train_ds.classes}")
print(f"Total images : {len(train_ds)} (Train) / {len(test_ds)} (Test)")

Classes chargées : ['apple_pie', 'french_fries', 'guacamole', 'hamburger', 'ice_cream', 'omelette', 'paella', 'pizza', 'sushi', 'tacos']
Total images : 2000 (Train) / 500 (Test)


---

### **Etape 1: Zero-Shot** 

1. Définir les prompts : Créer une liste de phrases comme "une photo de pizza".
2. Encoder le texte : Transformer ces phrases en vecteurs (embeddings) avec model.encode_text.
3. Comparer : Pour chaque image du set de test, calculer sa similarité avec tes 10 phrases. La phrase la plus proche donne la prédiction.
4. Calculer l'Accuracy : Comparer ces prédictions aux vrais labels.



#### 1. Préparation des prompts

In [7]:
# On récupère l'ordre exact des classes chargées par ImageFolder
class_names = test_ds.classes 
print(f"Ordre des classes : {class_names}")

# Création des prompts avec un template
templates = [f"a photo of {c}, a type of food" for c in class_names]
print(f"Exemple de prompt : {templates[0]}")

Ordre des classes : ['apple_pie', 'french_fries', 'guacamole', 'hamburger', 'ice_cream', 'omelette', 'paella', 'pizza', 'sushi', 'tacos']
Exemple de prompt : a photo of apple_pie, a type of food


#### 2. Encodage du texte

In [8]:
# Tokenisation et Encodage
text_tokens = clip.tokenize(templates).to(device)

with torch.no_grad():
    text_features = model.encode_text(text_tokens)
    # Normalisation pour le calcul de la similarité cosinus
    text_features /= text_features.norm(dim=-1, keepdim=True)

print(f"Dimensions des features texte : {text_features.shape}") # [10, 512]

Dimensions des features texte : torch.Size([10, 512])


#### 3. Comparaison & Calcul de l'Accuracy

In [9]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Évaluation Zero-shot"):
        images = images.to(device)
        labels = labels.to(device)

        # 1. Encoder l'image
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        # 2. Calculer la similarité (Produit scalaire car les vecteurs sont normalisés)
        # On multiplie par 100 (le logit_scale de CLIP) avant le softmax
        logits = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        
        # 3. Prendre la classe avec le score le plus haut
        preds = logits.argmax(dim=-1)
        
        correct += (preds == labels).sum().item()
        total += labels.size(0)

zero_shot_accuracy = (correct / total) * 100
print(f"\n--- RÉSULTAT ---")
print(f"Accuracy Zero-shot sur Food-10 : {zero_shot_accuracy:.2f}%")

Évaluation Zero-shot: 100%|██████████| 16/16 [00:05<00:00,  2.69it/s]


--- RÉSULTAT ---
Accuracy Zero-shot sur Food-10 : 98.80%


---

### **Etape 2: Extraction et Stockage des Features**

1. Passage unique : Tu passes toutes tes images de train_ds et test_ds dans model.encode_image.
2. Conversion NumPy : Tu stockes les résultats dans des tableaux NumPy (train_features et test_features).
Pourquoi ? Parce qu'entraîner un classifieur sur des vecteurs déjà extraits prend 1 seconde, alors que le faire sur des images prendrait des minutes.

#### 1. Fonction d'extraction des caractéristiques (Features)

In [10]:
import numpy as np

def extract_and_save_features(dataset, filename_prefix):
    features_list = []
    labels_list = []
    
    # Utilisation d'un batch_size de 32 pour ne pas saturer la RAM du Mac
    loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
    
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(loader, desc=f"Extraction {filename_prefix}"):
            # Envoi sur MPS (ou CPU)
            images = images.to(device)
            
            # 1. Passage dans l'encodeur d'images de CLIP
            features = model.encode_image(images)
            
            # 2. Normalisation L2 (essentielle pour la cohérence avec CLIP)
            features /= features.norm(dim=-1, keepdim=True)
            
            # 3. Stockage en NumPy (on quitte le GPU/MPS ici)
            features_list.append(features.cpu().numpy())
            labels_list.append(labels.numpy())
            
    # On concatène tous les batches
    final_features = np.concatenate(features_list, axis=0)
    final_labels = np.concatenate(labels_list, axis=0)
    
    # Sauvegarde sur le disque
    np.save(f"{filename_prefix}_features.npy", final_features)
    np.save(f"{filename_prefix}_labels.npy", final_labels)
    
    return final_features, final_labels


#### 2. Exécution et Stockage

In [11]:
print("🚀 Début de l'extraction massive...")

# Extraction pour le Train (200 images par classe)
train_features, train_labels = extract_and_save_features(train_ds, "train")

# Extraction pour le Test (50 images par classe)
test_features, test_labels = extract_and_save_features(test_ds, "test")

print("\n✅ Extraction terminée et fichiers sauvegardés !")
print(f"Dimensions Train : {train_features.shape}") # Devrait être [2000, 512]
print(f"Dimensions Test : {test_features.shape}")   # Devrait être [500, 512]

🚀 Début de l'extraction massive...


Extraction test: 100%|██████████| 16/16 [00:04<00:00,  3.36it/s]


✅ Extraction terminée et fichiers sauvegardés !
Dimensions Train : (2000, 512)
Dimensions Test : (500, 512)


### Etape 3: Scénarios Linear Probing (1-shot et 5-shots)

1. Sélection des données (le "N-shot") :
    - Pour le 1-shot : Tu parcours ton set d'entraînement et tu prends exactement 1 image au hasard pour chacune des 10 classes.
    - Pour le 5-shots : Tu prends 5 images par classe.

2. Entraînement : Tu utilises une LogisticRegression (de Scikit-Learn) ou une couche Linear (PyTorch). Tu l'entraînes à faire le lien entre les features extraites à l'étape 2 et les labels.

3. Évaluation : Tu testes ce classifieur sur tes test_features.

### Etape 4: Evaluation 